In [1]:
import numpy as np
import pandas as pd
import re
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

In [2]:
utts = pd.read_csv('data/utts.csv')
roberts = pd.read_csv('data/subset.csv')

In [3]:
utt_counts = pd.DataFrame(utts.groupby(['meta.case_id']).size())
utt_counts = utt_counts.reset_index()
utt_counts = utt_counts.rename(columns={0: 'utt_counts'})
utts = pd.merge(utts, utt_counts, how = 'left')
utts = utts.loc[utts.loc[:, 'utt_counts'] != 1, :]
utts.groupby(['meta.case_id']).size()

utts['text'].replace('\d+', '', regex=True, inplace = True) 
#remove numbers, decide as group if this makes sense
#NOTE: No stemming or lemmatization done at this point
convo_case = pd.DataFrame(utts.groupby('meta.case_id')['conversation_id'].nunique())
convo_case.rename(columns = {'conversation_id':'convo_count'}, inplace = True)
#Code to create conversations/case feature

group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')

X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != 'win_side'],df['win_side'], test_size = 0.2, random_state = 0)
convo_case


,convo_count
meta.case_id,
2005_03-1238,1
2005_04-1034,1
2005_04-10566,1
2005_04-1067,1
2005_04-1084,1
...,...
2019_19-631,1
2019_19-635,1
2019_19-67,1


In [20]:
count_vect = CountVectorizer(ngram_range = (1,1),min_df = 4,
            stop_words = 'english', token_pattern = r'\b[a-zA-Z]{3,}\b') 
count_df = count_vect.fit_transform(X_train['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = count_vect.get_feature_names(), index = X_train['id'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(count_df).toarray()
X_train_tfidf = pd.DataFrame(X_train_tfidf, columns = count_vect.get_feature_names(), index = X_train['id'])

X_train_tfidf = pd.merge(X_train_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
#MERGING IN ADDITIONAL FEATURES HERE
X_train_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abbott,...,younger,youth,youthful,zero,zillion,zip,zone,zones,zoning,convo_count
id,,,,,,,,,,,,,,,,,,,,,
2015_15-290,0.000000,0.000000,0.007653,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.120762,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2007_07-343,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.006524,0.007268,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2009_09-497,0.006651,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2005_04-1131,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.009423,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.006984,0.0,0.0,1
2009_09-338,0.000000,0.008398,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016_16-6219,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2007_06-984,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.003476,0.0,0.0,0.000000,0.0,0.0,1
2013_12-79,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.003816,0.0,0.0,0.000000,0.0,0.0,1


In [21]:
count_test = count_vect.transform(X_test['text'])
count_test_array = count_test.toarray()
count_test_df = pd.DataFrame(count_test_array, columns = count_vect.get_feature_names(),index = X_test['id'])

X_test_tfidf = tfidf_transformer.transform(count_test_df).toarray()
X_test_tfidf = pd.DataFrame(X_test_tfidf, columns = count_vect.get_feature_names(), index = X_test['id'])
X_test_tfidf = pd.merge(X_test_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
X_test_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abbott,...,younger,youth,youthful,zero,zillion,zip,zone,zones,zoning,convo_count
id,,,,,,,,,,,,,,,,,,,,,
2016_15-649,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2005_04-1544,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2006_05-1074,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2013_12-158,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.004832,0.0,0.0,1
2009_08-1553,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015_14-419,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.004816,0.0,0.0,0.000000,0.0,0.0,1
2009_08-1529,0.0,0.0,0.0,0.005103,0.021409,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1
2009_09-475,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1


In [22]:
model = GaussianNB() 
model.fit(X_train_tfidf,y_train)
win_pred = model.predict(X_test_tfidf)

print(accuracy_score(y_test,win_pred))
"""
this accuracy is notably worse than the 'just say the petitioner will win 
every time and you'll be right 65% of the time'
"""
print(precision_score(y_test,win_pred))

0.6292682926829268
0.644808743169399
